## Download Resourses

In [ ]:
link = 'http://crisisnlp.qcri.org/data/lrec2016/labeled_cf/CrisisNLP_labeled_data_crowdflower_V1.2.zip'
dest = '../../data/Datasets'
datasets = 'CrisisNLP_labeled_data_crowdflower_V1.2.zip'    

In [ ]:
!cd $dest && wget $link && unzip $datasets && rm -r $datasets

In [ ]:
original_file = '2014_california_eq.csv'
original_folder = '../../data/Datasets/CrisisNLP_labeled_data_crowdflower.DS_Store/2014_California_Earthquake'
dest_folder = '../../data/Datasets/Training_Data/California-Quake'

In [ ]:
!cd $dest_folder && awk -F ',' '{print>$1}' $original_file

In [ ]:
!cd $original_folder && awk -F ',' '{print > ("data"$0".csv")}' $original_file

In [ ]:
!cat $original_file

In [ ]:
!java -classpath TweetsRetrieval-1.2-jar-with-dependencies.jar qa.qcri.tweetsretrieval.TweetsRetrievalTool $original_file 

## Import some nltk functions

In [1]:
import nltk
import csv
import string
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def expand_around_chars(text, characters):
    for char in characters:
        text = text.replace(char, ' ' + char + ' ')
    return text

def strip_quotations_newline(text):
    clean_tweet = ' '.join(text.split())
    clean_tweet = clean_tweet.decode('unicode_escape').encode('ascii','ignore')
    clean_tweet = clean_tweet.replace('",\'','')
    clean_tweet = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", clean_tweet)
    clean_tweet = re.sub(r'''(@[A-Za-z0-9]+)''', "", clean_tweet)
    clean_tweet = re.sub("([0-9]+)", "", clean_tweet)
    return clean_tweet

def split_text(text):
    text = strip_quotations_newline(text)
    text = expand_around_chars(text, '\/".,()&[]{}:;!-_\'')
    splitted_text = text.split(' ')
    cleaned_text = [x for x in splitted_text if len(x) > 1]
    text_lowercase = [x.lower() for x in cleaned_text]
    text_stemmed = [stemmer.stem(x) for x in text_lowercase]
    text_stemmed = [x.decode('unicode_escape').encode('ascii','ignore') for x in text_stemmed]
    text_filtered = [x for x in text_stemmed if not x in stop_words]    
    return text_filtered

## JSON to CSV

In [6]:
import glob
import os
import json

In [4]:
#path = '/home/user/elicon/data/Test_Data/Nepal_Quake/json/batch2'
path = '/home/user/tweets.json'
os.chdir(path)

input_files = glob.glob("*.csv")
for finput in input_files:
    with open(finput) as fin:
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".ttxt", "w+") as fout:
            for line in fin:
                try:
                    tweet = json.loads(line)
                    id = tweet['created_at']
                    if 'retweeted_status' in tweet:
                        text = tweet['retweeted_status']['text']
                    else:
                        text = tweet['text']
                    fout.write(str(id) + "," + str(text) + "\n")
                except:
                    continue

OSError: [Errno 20] Not a directory: '/home/user/tweets.json'

In [8]:
dataset = '/home/user/tweets_3.json'
with open(dataset) as fin:
        with open(( dataset.rsplit( ".", 1 )[ 0 ] ) + ".csv", "w+") as fout:
            for line in fin:
                try:
                    tweet = json.loads(line)
                    id = tweet['created_at']
                    if 'retweeted_status' in tweet:
                        text = tweet['retweeted_status']['text']
                    else:
                        text = tweet['text']
                    fout.write(str(id) + "," + str(text) + "\n")
                except:
                    continue

## Json to Csv keeping time and location information

In [3]:
from datetime import datetime
import time

import json, glob, os
#path = '/home/user/elicon/data/Test_Data/Chile_Quake/unlabelled/'
path = '/home/user/datasets'
os.chdir(path)
os.getcwd()
input_files = glob.glob("*.json")

#dataset = '/home/user/datasets/tweets.json'
for finput in input_files: #iterates over all files in the directory ending in .csv
    with open(finput) as fin:
#with open(dataset) as fin:
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".csv", "w+") as fout:
            for line in fin:
                try:
                    tweet = json.loads(line)
                    date = tweet['created_at']
                    try:
                        c = time.strptime(date.replace("+0000",''), '%a %b %d %H:%M:%S %Y')
                    except: 
                        print "pb with tweet_gmttime", date, line
                        pass

                    tweet_unixtime = int(time.mktime(c))
                    date_time = datetime.fromtimestamp(tweet_unixtime).strftime('%Y-%m-%d %H:%M:%S')

                    if tweet['place']:
                        location = tweet['place']
                    else:
                        location = tweet['user']['location']
                    if 'retweeted_status' in tweet:
                        text = tweet['retweeted_status']['text']
                    else:
                        text = tweet['text']
                    filtered_text = ' '.join(split_text(text))    
                    fout.write(str(date_time) + ',' + str(tweet_unixtime) + ',' + str(location) + "," + str(filtered_text) + "," + str(text) + "\n")
                except:
                    continue

In [4]:
spark

In [20]:
path = '/home/user'
os.chdir(path)

In [4]:
!awk -F ',' -v OFS=',' '{print $2}' tweets_3.csv > tweets_3.csvv 

In [5]:
!sed 's/http[^ ]*//g' tweets_3.csvv > tweets_3.csv1

In [6]:
!awk '!seen[$0]++' tweets_3.csv1 > tweets_3.csv2

In [9]:
!awk 'NF>=3' tweets_2.csv1 > tweets_2.csv

## Stemming,Stop words removing and other normalisations

In [10]:
import json
#path = '/home/user/elicon/data/Test_Data/Chile_Quake/unlabelled/'
path = '/home/user/elicon/data/'
os.chdir(path)
os.getcwd()
input_files = glob.glob("*.csv")
for finput in input_files: #iterates over all files in the directory ending in .csv
    with open(finput) as fin:
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".csv1", "w+") as fout:
            for tweet in fin:
                clean_doc = ' '.join(split_text(tweet))
                json.dump(clean_doc, fout)
                fout.write("\n")

In [5]:
#finput = '/home/user/elicon/data/Test_Data/Chile_Quake/unlabelled/chile_quake_rawTweets.txt'
#finput = '/home/user/elicon/data/Test_Data/California_Quake/datasets'
finput = '/home/user/tweets.xcsv'
with open(finput) as fin:
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".csv", "w+") as fout:
            fout.write('tweet_text,label\n')
            for tweet in fin:
                clean_doc = ' '.join(split_text(tweet))
                json.dump(clean_doc, fout)
                fout.write("\n")

## Add labels for multiclass classification 

In [ ]:
## add labels 
path = '/home/user/elicon/data/Training_Data/Nepal_Quake/labeled'
os.chdir(path)
#os.getcwd()
input_files = glob.glob("*.csv")
for finput in input_files: #iterates over all files in the directory ending in .csv
    with open(finput, 'r') as fin:
        if finput == 'injured_or_dead_people.csv':
            string_to_add = ", 0"
        elif finput == 'missing_trapped_or_found_people.csv':
            string_to_add = ", 1"
        elif finput == 'displaced_people_and_evacuations.csv':
            string_to_add = ", 2"
        elif finput == 'infrastructure_and_utilities_damage.csv':
            string_to_add = ", 3"
        elif finput == 'donation_needs_or_offers_or_volunteering_service.csv':
            string_to_add = ", 4"
        elif finput == 'caution_and_advice.csv':
            string_to_add = ", 5"
        elif finput == 'sympathy_and_emotional_support.csv':
            string_to_add = ", 6"
        elif finput == 'other_useful_information.csv':
            string_to_add = ", 7"
        elif finput == 'not_related_or_irrelevant.csv':
            string_to_add = ", 8"
        else:
            string_to_add = ", 9"

        file_lines = [''.join([x.strip(), string_to_add, '\n']) for x in fin.readlines()]
        
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".txt", "w+") as fout:
            fout.write('tweet_text,label\n')
            fout.writelines(file_lines)

## Equisample lines from all classes

In [ ]:
import random
path = '/home/user/elicon/data/Training_Data/Binary_Classification_Data/Nepal_Quake'
os.chdir(path)
dest_file = 'train.csv'

input_files = glob.glob("*.txt")

with open(dest_file, "a") as sink:
    sink.write('tweet_text,label\n')
    for finput in input_files:
        with open(finput, 'r') as fin:
            lines = [line for line in fin]
            random_choice = random.sample(lines, 2800)            
            sink.write("".join(random_choice))

## Add labels for binary classification 

In [11]:
#path = '/home/user/elicon/data/Training_Data/Binary_Classification_Data/Nepal_Quake'
path = '/home/user/elicon/data'
os.chdir(path)

input_files = glob.glob("*.csv1")
for finput in input_files: #iterates over all files in the directory ending in .csv
    with open(finput, 'r') as fin:
        if finput == 'negative.csv1':
            string_to_add = ", 0"
        elif finput == 'positive.csv1':
            string_to_add = ", 1"
        else:
            string_to_add = ", 2"
        file_lines = [''.join([x.strip(), string_to_add, '\n']) for x in fin.readlines()]
        
        with open(( finput.rsplit( ".", 1 )[ 0 ] ) + ".txt", "w+") as fout:
            #fout.write('tweet_text,label\n')
            fout.writelines(file_lines)

## Equisample lines form both positive and negative class

In [12]:
import random
#path = '/home/user/elicon/data/Training_Data/Binary_Classification_Data/Nepal_Quake'
#path = '/home/user/elicon/data/Test_Data/California_Quake/datasets/'
path = '/home/user/elicon/data'
os.chdir(path)
dest_file = 'train.csv'

input_files = glob.glob("*.txt")

with open(dest_file, "a") as sink:
    sink.write('tweet_text,label\n')
    for finput in input_files:
        with open(finput, 'r') as fin:
            lines = [line for line in fin]
            random_choice = random.sample(lines, 400)            
            sink.write("".join(random_choice))
            

ValueError: sample larger than population

In [ ]:
import json
from datetime import datetime
import pandas as pd
import time
import string

tweets_data_path = r'/home/user/Gmail/nepal_eq/json/idinfrastructure_and_utilities_damage.json'
tweets_dest = r'/home/user/elicon/data/Training_Data/Nepal_Quake/infrastructure_and_utilities_damage.csv'
tweets_data = []
with open(tweets_data_path, "r") as tweets_file:
    with open(tweets_dest,'w+') as fout:
        for line in tweets_file:
            try:
                tweet = json.loads(line)
                id = tweet['id']
                if 'retweeted_status' in tweet:
                    text = tweet['retweeted_status']['text']
                else:
                    text = tweet['text']
                fout.write(str(id) + "," + str(text) + "\n")
            except:
                continue

## Create training datasets as pairs tweet and label

In [ ]:
## create the tweet label file
original_file = "../../data/California-Earthquake/califor_neg.csv"
tweet_file = "../../data/California-Earthquake/califor_neg_sem.csv"
output_file = "../../data/California-Earthquake/califor_neg_train.csv"
!cat $original_file | cut -d , -f2 > $tweet_file

In [ ]:
import json
filtr_file = "../../data/California-Earthquake/califor_neg_filtr.csv"
with open(tweet_file, "r") as fin,open(filtr_file, "w+") as fout:
    for tweet in fin:
        clean_doc = ' '.join(split_text(tweet))
        json.dump(clean_doc, fout)
        fout.write("\n")

In [ ]:
h

!rm $tweet_file
!rm $filtr_file

## Add a header to the training file

In [ ]:
import csv, glob

path = '/home/user/elicon/data/Training_Data/Nepal_Quake/labeled'
os.chdir(path)
#os.getcwd()
input_files = glob.glob("*.csv")
for training_file in input_files: #iterates over all files in the directory ending in .csv
    with open(training_file, "wb") as train_file:
        writer = csv.writer(train_file)
        writer.writerow(["tweet_text", "label"])

## Shuffle lines to get the training dataset

In [ ]:
with open(dest_file, "rb") as source:
    lines = [line for line in source]
import random
random_choice = random.sample(lines, 200)
with open(training_file, "a") as sink:
    sink.write("".join(random_choice))
!rm $dest_file

## Extract time

In [ ]:
from datetime import datetime
import time

date = tweet['created_at']
try:
    c = time.strptime(tweet_gmttime.replace("+0000",''), '%a %b %d %H:%M:%S %Y')
except: 
    print "pb with tweet_gmttime", tweet_gmttime, line
    pass

tweet_unixtime = int(time.mktime(c))
